In [1]:
import sys, os, time

In [37]:
import scipy.stats as stats

In [2]:
import pandas as pd
import numpy as np

In [3]:
from glob import glob

In [4]:
g = glob('vi_data/*.csv')

In [7]:
x = pd.read_csv(g[0], header=None)

In [13]:
def parse_single_line_csv(fname, sep=','):
    with open(fname, 'r') as f:
        row = f.readline()
        row = row.split(sep)
        cl_meta = []
        for i in range(2):
            cl_fname = row[i]
            spl = cl_fname.split('/')
            method = spl[4]
            cl_id = spl[5]
            cl_meta.append( (method, cl_id) )
        cl_meta = tuple(cl_meta)
        vi = float(row[2])
    return cl_meta, vi

In [14]:
parse_single_line_csv(g[0])

((('undirdir', '1160295_235'), ('outdirdir', '1166684_245')), 1.83587110988)

In [15]:
(((x1, x2), (y1,y2)), vi) = parse_single_line_csv(g[0])

In [17]:
print(x1, x2, y1, y2, vi)

('undirdir', '1160295_235', 'outdirdir', '1166684_245', 1.83587110988)


In [21]:
rows = []
for fname in g:
    (((x1, x2), (y1,y2)), vi) = parse_single_line_csv(fname)
    rows.append( (x1, x2, y1, y2, vi) )
    

In [27]:
columns = ['method_1', 'id_1', 'method_2', 'id_1', 'vi']
df = pd.DataFrame(rows, columns=columns)

In [28]:
def methods_parse(row):
    methods = [row['method_1'], row['method_2']]
    methods.sort()
    return ', '.join(methods)
df['methods'] = df.apply(methods_parse, axis=1)

In [46]:
gb = df.groupby('methods')
gb['vi'].mean().sort_values()

methods
pagerank, pagerank                          1.176754
pagerank_unrecorded, pagerank_unrecorded    1.799974
undirdir, undirdir                          1.862236
pagerank_unrecorded, undirdir               1.875018
outdirdir, outdirdir                        1.884981
outdirdir, pagerank_unrecorded              1.944723
outdirdir, undirdir                         1.970177
pagerank, undirdir                          2.002739
outdirdir, pagerank                         2.061885
pagerank, pagerank_unrecorded               2.086319
Name: vi, dtype: float64

In [47]:
values_per_group = [col for _, col in gb['vi']]

In [52]:
stats.f_oneway(*values_per_group)

F_onewayResult(statistic=168.96005840355954, pvalue=5.8831001346532458e-86)

In [62]:
stats.ttest_ind(df[df.methods=='pagerank_unrecorded, pagerank_unrecorded']['vi'], df[df.methods=='pagerank_unrecorded, undirdir']['vi'])

Ttest_indResult(statistic=-3.3068230486446049, pvalue=0.0027610528858429172)

In [ ]:
stats.ttest_ind